# 議事録アプリ実装体験

## 免責事項
> これは議事録アプリを0から作成するハンズオンWS用ファイルであり、その目的に特化して作成されています。このファイルを使用した際に発生する可能性のあるいかなる不具合についても責任を負いかねます。<br>ファイルの品質と機能性を確保するために万全を尽くしておりますが、万一の不具合発生時にはご理解いただけますようお願い申し上げます。

## 操作方法

下記に用意されているコードに対して、<br>
> windows: コードブロックを選択した状態でCtrl + Enter<br>
>Mac: コードブロックを選択した状態でCommand + Enter

もしくは▶マークを選択して、コードを起動してください。

## 確認

コードブロック横が✔になれば動作は完了しています。<br>
Warningが表示されていたとしても、先に続くコードが活用できていれば基本問題はございません。

In [ ]:
##################### シャープはコメントアウトと呼び、コード部分ではない解説やメモを記載できます #######################

# 議事録アプリを作ろう！
議事録を作るには、
①動画や音声データから文字を書き起こし
②書き起こしデータから、LLMを使って用途に合わせて要約
③要約データを保存する
という３ステップで実装することができます。

## ○目標
*   書き起こし等の様々なライブラリを使ってみる
*   アプリケーション構築の一連の流れを体験する

## ○体験できること：
*   Pythonとライブラリを活用したアプリケーション作成
*   Speech to Textサービス、Whisperを用いた文字起こし
*   OpenAIのGPTを使ったプロンプトエンジニアリング
*   Pandasを用いたデータ整理






# 書き起こしをやってみよう

## 目的
moviepyとwhisperを用いて、動画から音声に、音声から文字起こし、を体験してみましょう。

*   ライブラリの使用方法を学ぶ
*   文字起こし機能を実装する


Whisperとは？
https://platform.openai.com/docs/guides/speech-to-text
https://github.com/openai/whisper

Whisperで使えるモデルを学ぼう
https://qiita.com/diesekiefer/items/00d8c1507829b58a62ab


Whisperを使って動画ファイルをテキスト化しよう
https://platform.openai.com/docs/guides/speech-to-text/quickstart


## 動画ファイルから音声ファイルに変換

In [4]:
# githubというコード共有・管理サイト上にある、テスト用のmp4ファイルをダウンロードします（Google corabのクラウド上へ）
# https://github.com/jidansan6/testmovie/blob/328b381957c76484894d4dbb7d9628c8b50c5c5d/testmovie.mp4 -debug用
!wget https://github.com/jidansan6/testmovie/raw/328b381957c76484894d4dbb7d9628c8b50c5c5d/testmovie_ma.mp4 -O testmovie.mp4

--2024-07-29 15:33:11--  https://github.com/jidansan6/testmovie/raw/328b381957c76484894d4dbb7d9628c8b50c5c5d/testmovie_ma.mp4
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/jidansan6/testmovie/328b381957c76484894d4dbb7d9628c8b50c5c5d/testmovie_ma.mp4 [following]
--2024-07-29 15:33:11--  https://raw.githubusercontent.com/jidansan6/testmovie/328b381957c76484894d4dbb7d9628c8b50c5c5d/testmovie_ma.mp4
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 25016147 (24M) [application/octet-stream]
Saving to: ‘testmovie.mp4’

testmovie.mp4       100%[===================>]  23.86M  --.-KB/s    in 0.08s   

20

Pythonは①豊富なライブラリが存在すること、➁学習者が多く、情報も多いことから、人気なプログラミング言語のひとつです。ライブラリを組み合わせてアプリを作成してみましょう。

In [5]:
# 必要なライブラリのインストール（機能をネットワーク上から、今回の作業環境へ引っ張ってくる）
# たとえるならば、ドラゴンクエストの武器屋で武器を「購入」するイメージ（未装備）
!pip install moviepy

In [6]:
# この環境で使えるようにする（装備的な感覚）
# 初期装備として、pip installが不要なライブラリも存在します（時間を制御するtimeや、jsonファイルを取りあつかうjson,requestなど）
from moviepy.editor import VideoFileClip

In [7]:
# 処理するファイルを選択します
video_file_path = "testmovie.mp4"

In [8]:
# ビデオファイルを「moviepy」の機能で読み込みます（clipの中に、対象のmp4を動画で格納しているイメージです）
clip = VideoFileClip(video_file_path)

In [9]:
# ビデオからオーディオを抽出し、wavファイルとして保存する
clip.audio.write_audiofile("audio.wav", 44100, 2, 2000,"pcm_s32le")

MoviePy - Writing audio in audio.wav


MoviePy - Done.


これで動画から音声のみを抽出することができました。Whisperを使って、文字起こしをしてみましょう。

In [10]:
# まずはwhisperの機能をインストール＋装備します。
!pip install git+https://github.com/openai/whisper.git
import whisper

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-2q_sa4y3
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-2q_sa4y3
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)


In [11]:
# 早速使ってみましょう。まずはtinyモデルを使用してみます
model_tiny = whisper.load_model("tiny")
result_tiny = model_tiny.transcribe("audio.wav")
print(result_tiny["text"])

100%|█████████████████████████████████████| 72.1M/72.1M [00:17<00:00, 4.34MiB/s]


ないですか?やめったりなわないですけどもこのまま帰りたいなともことがあるのか?はい最初からいたい4段今回は金融方々に職されているパーダシーやさんにお越しいときましたパーダさんよろしくお願いしますよろしくお願いします普通ですねいつも通りですね全員緊張しなさそう全員緊張してますしてますしてますよろしくお願いしますパーダさんは今ニューシャーな年目なりますか?2020年の2月に入りしたので水分かげつ10ヶ月とかそんな感じです今紹介したのかで一番任せてですね10ヶ月1年見つない10ヶ月のターダさん全職って何されてましたか?全職は新たく銀行に2回した新たく銀行お仕事どんなことされてたんですか?新たく銀行では個人の客様を相手に学業相談冬さん有以本みたいなそんな話を4ヶ月をやらせていった大手その後法人へ行っていうことで結構大企業のオリアゲでいくと100億とかそれぐらいを超える客様を多分としてこういう仕方にとかいわゆる法人マーリーのご相談みたいなそんなことをやってましたすごく高生石を覚められていたって言わさを聞いてますよくねも全然人に僕でポジトになりますはいそんなターダさんですがどうしてこのM&Aセンターに入ろうと思われたんですかもっともっと新たく銀行に入ったのも個人と保持人両方をやって両方のお悩みに乗れるようなまあ車員というかこういうのがになりたいと思って入ってでもそれやっていく中でやっぱり新たく銀行がやる保持人へ行って大企業が相手なので素敵大素敵っていうことでやっぱり大なというか経営者とちょっと距離が当鳴られていることを感じたりしながらなんか中小企業のエマンドエみたいなところだと大なさんの保持人の話にもそうなのれてでかつ会社のお悩みにも公園できるっていうところはいいなと思ってはい入者受けましたはいなるほど本当はお金とかじゃないですか金もまあ今さらになんかもないやればやっただけその評価されるみたいなその体系っていうんですかそういうのはやっぱり魅力には思ったとこでも持ちろんありますそうですねよねやっぱそこがうちの会社の大好みかなっていうふうに私も思いますとお仕事を気付いているような言われてますけれども一番辛かったエピソードなどあれば教えていただけますかえーとまだ入者してそんなに期間もないので自分辛かったなというのもお頃がましみんですがやっぱり一番最初の件をやらせていただいた時に今今今

文字起こしが完了しましたね！
違うモデルも試してみましょう。

In [12]:
# もう少し高精度なsmallモデルを使用してみます
model_small = whisper.load_model("small")
result_small = model_small.transcribe("audio.wav")
print(result_small["text"])

100%|███████████████████████████████████████| 461M/461M [00:05<00:00, 84.0MiB/s]


ないですかやめたいなはないですけどもこのまま変わりたいなと思うことは最初から第4弾今回は金融法人部に所属されているた原信仰さんにお越しいただきましたた原さんよろしくお願いしますよろしくお願いします普通ですねいつも通りですね全然緊張しなさそうですめっちゃ緊張してますしてますしてますしてますお願いしますた原さんは今入社何年目になりますか 2020年の2月に入社したので9か月10か月とかそんな感じです今紹介した中で一番若手ですね10か月1年もいない 10か月のた原さんはい全職って何されてましたか 全職は新宅銀行にいました新宅銀行お仕事どんなことされてたんですか新宅銀行新宅銀行では個人のお客様は1点に運用相談不動産有意言みたいなそんな話を4年半ほどやらせていただいてその後法人営業ということで結構大企業の売上でいくと100億とかそれぐらいを超えるお客様を担当して保有しだったりとか まあいわゆる法人周りのご相談みたいなそんなことをやってましたすごく高成績を収められていたという話を聞いてます全然人に怒れることがないですそんなた原さんですがどうしてこのエマンドエーセンターに入ろうと思われたんですかもっともっと新宅銀行に入ったのも個人と法人両方やって両方のお悩みに乗れるような社員というか公演になりたいと思って入ってそれをやっていく中でやっぱり新宅銀行がやる法人営業って大企業が相手なので組織対組織っていうことでやっぱりオーナーというか経営者とちょっと距離が遠いなーなんていうことを感じたりしながら中小企業のエマンドエーみたいなところだとオーナーさんの個人の話にも相談乗れてかつ会社のお悩みにも貢献できるというところがいいなと思って入社を受けましたなるほどー本当はお金とかじゃないですかお金もまあやればやっただけ評価されるみたいな体系というんですかそういうのはやっぱり魅力には思ったところももちろんありますそうですよねやっぱそこがうちの会社の大ゴミかなというふうに私も思いますお仕事をきついていろいろ言われてますけれども一番辛かったエピソードなどあれば教えていただけますかまだ入社してそんなに期間もないので一番辛かったなんていうのも小川真ですがやっぱり一番最初の案件をやらせていただいたときにまあエマンドエーというか株式上等ということで株を渡しするのが基本なんですけれども事業上等ということで事業を移す

In [13]:
# 比較するとsmallの方がより高精度であることが想像できます。一方で処理時間については高精度の方が長くなることがわかりました。
print(result_tiny["text"])
print(result_small["text"])

ないですか?やめったりなわないですけどもこのまま帰りたいなともことがあるのか?はい最初からいたい4段今回は金融方々に職されているパーダシーやさんにお越しいときましたパーダさんよろしくお願いしますよろしくお願いします普通ですねいつも通りですね全員緊張しなさそう全員緊張してますしてますしてますよろしくお願いしますパーダさんは今ニューシャーな年目なりますか?2020年の2月に入りしたので水分かげつ10ヶ月とかそんな感じです今紹介したのかで一番任せてですね10ヶ月1年見つない10ヶ月のターダさん全職って何されてましたか?全職は新たく銀行に2回した新たく銀行お仕事どんなことされてたんですか?新たく銀行では個人の客様を相手に学業相談冬さん有以本みたいなそんな話を4ヶ月をやらせていった大手その後法人へ行っていうことで結構大企業のオリアゲでいくと100億とかそれぐらいを超える客様を多分としてこういう仕方にとかいわゆる法人マーリーのご相談みたいなそんなことをやってましたすごく高生石を覚められていたって言わさを聞いてますよくねも全然人に僕でポジトになりますはいそんなターダさんですがどうしてこのM&Aセンターに入ろうと思われたんですかもっともっと新たく銀行に入ったのも個人と保持人両方をやって両方のお悩みに乗れるようなまあ車員というかこういうのがになりたいと思って入ってでもそれやっていく中でやっぱり新たく銀行がやる保持人へ行って大企業が相手なので素敵大素敵っていうことでやっぱり大なというか経営者とちょっと距離が当鳴られていることを感じたりしながらなんか中小企業のエマンドエみたいなところだと大なさんの保持人の話にもそうなのれてでかつ会社のお悩みにも公園できるっていうところはいいなと思ってはい入者受けましたはいなるほど本当はお金とかじゃないですか金もまあ今さらになんかもないやればやっただけその評価されるみたいなその体系っていうんですかそういうのはやっぱり魅力には思ったとこでも持ちろんありますそうですねよねやっぱそこがうちの会社の大好みかなっていうふうに私も思いますとお仕事を気付いているような言われてますけれども一番辛かったエピソードなどあれば教えていただけますかえーとまだ入者してそんなに期間もないので自分辛かったなというのもお頃がましみんですがやっぱり一番最初の件をやらせていただいた時に今今今

In [14]:
# 文字起こし結果をテキストファイルに一応保存しておきましょう。右側のフォルダ更新マークを押して確認してみましょう。
output_file_path = 'result.txt'
with open(output_file_path, 'w') as file:
    file.write(result_small["text"])

無事に動画を文字起こしすることができました！

もちろんこれらのコードは0から書く必要はなく、「誰かが作ってくれたライブラリ≒機能」をインストール→装備して使うイメージになります。必要なところを書き換える、ライブラリ同士を組み合わせるというのが現在のアプリ開発の主流となっています。

結果を見てみると、話している人の区別がついていないので、議事録を作成するには話者分離を行う必要がありそうです。<br>
→今回は時間の都合上割愛しますが、Langchainなどのライブラリを活用すると、話者分離も可能です。

# 文字起こし文章の要約

## 目的
文字起こし文章のテキストデータをGPTの機能を使ってで要約する中で、下記の知識を身につける。
* GPTをAPIで呼び出し、生成AIを使う第一歩を踏み出してみましょう。
* プロンプトの工夫点を学び、自分なりにプロンプトを書けるようになる

In [15]:
# 今度はAzure上のOpenAI GPTを呼び出せるようにライブラリをインストールします。
!pip install openai==0.27.8
import openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 5.3 MB/s eta 0:00:00


In [16]:
openai.api_type = "azure"
openai.api_base = "https://tech0-gpt-event-westus.openai.azure.com"
# 実際はこのように書くと、前述の通り悪用の可能性があります→環境変数として別保存が望ましいですが今回は割愛します。
openai.api_key = "8db5e51a08ab497a9e31003e67b23a5d"
openai.api_version = "2023-05-15"

In [17]:
# chatGPTに質問する
response = openai.ChatCompletion.create(
    engine="gpt-4o",
    messages=[
        {"role": "user", "content": "大谷翔平について50文字以内で教えて"},
    ],
)
# 回答の表示
print(response.choices[0]["message"]["content"].strip())

大谷翔平は日本のプロ野球選手で、投打両方で活躍する「二刀流」として有名。MLBで活躍中。


意外と簡単に読み出せましたね！

In [18]:
# 好きな内容に変えてみましょう。
response = openai.ChatCompletion.create(
    engine="gpt-4o",
    messages=[
        {"role": "user", "content": "ここに質問を記載します"},
    ],
)
# 回答の表示
print(response.choices[0]["message"]["content"].strip())

もちろんです。質問をどうぞ。この場で可能な限りお答えいたします。


### 文字起こし文章の確認
文字起こし＆要約アプリの打ち合わせをした時の文字起こしデータを要約してみましょう。

ただし、扱える文章量には制限があることに注意します。今回は短めの文章なので問題なく対応できるはずです。

トークン数のカウント：https://techblog.gmo-ap.jp/2023/06/12/tiktoken/

In [19]:
# テキストデータをファイルから開いて、変数に格納（変数all_text）
with open("result.txt") as f:
    all_text = f.read()

# 文字数の確認
print(str(len(all_text))+" 文字")

#文章の一部を表示
print("---------------------")
print(all_text[0:100])

3483 文字
---------------------
ないですかやめたいなはないですけどもこのまま変わりたいなと思うことは最初から第4弾今回は金融法人部に所属されているた原信仰さんにお越しいただきましたた原さんよろしくお願いしますよろしくお願いします普通


In [20]:
# それでは、要約してみましょう！
# f～というのは、中身に変数が入っていますよ、という宣言です。
def ai_summary(text):
    response = openai.ChatCompletion.create(
        engine="gpt-4o",
        messages=[
            {"role": "system", "content": "下記の文章は文字起こしのテキストファイルです。\
            「あー」や、「えーっと」などに代表される、つなぎ言葉を削除して校正してください。"},
            {"role": "user", "content": text}
        ],
    )
    # 回答の表示
    res = response.choices[0]["message"]["content"].strip()
    return res

In [21]:
# 使ってみましょう！
res = ai_summary(all_text)
print(res)

ないですかやめたいなはないですけどもこのまま変わりたいなと思うことは最初から第4弾今回は金融法人部に所属されている田原信仰さんにお越しいただきました。田原さんよろしくお願いします。

よろしくお願いします。

普通ですね。いつも通りですね。全然緊張しなさそうです。

めっちゃ緊張してます。

お願いします。田原さんは今入社何年目になりますか？

2020年の2月に入社したので9か月、10か月とかそんな感じです。

今紹介した中で一番若手ですね。

10か月、1年もいない。10か月の田原さん。全職って何されていましたか？

全職は新宅銀行にいました。

新宅銀行。お仕事どんなことされてたんですか？

新宅銀行では個人のお客様に運用相談、不動産、有意言みたいな話を4年半ほどやらせていただいて、その後法人営業ということで結構大企業の売上が100億とかそれぐらいを超えるお客様を担当して、法人周りのご相談をやっていました。

すごく高成績を収められていたという話を聞いています。

全然そんなことないです。

どうしてこのエマンドエーセンターに入ろうと思われたんですか？

もっともっと新宅銀行に入ったのも、個人と法人両方やって両方のお悩みに乗れるような社員になりたいと思って入って、それをやっていく中で新宅銀行がやる法人営業って大企業が相手なので組織対組織ということで、オーナーや経営者と距離が遠いなと感じて、中小企業のエマンドエーみたいなところはオーナーさんの個人の話にも相談乗れて、かつ会社のお悩みにも貢献できるところがいいなと思って入社を受けました。

なるほど、お金とかじゃないですか？

お金もやればやっただけ評価されるみたいな体系は魅力に思った部分もあります。

うちの会社の大魅力かなと思います。お仕事をしている中で一番辛かったエピソードなどあれば教えていただけますか？

まだ入社してそんなに期間もないので一番辛かったとは言えませんが、一番最初の案件をやらせていただいたときに株式上等ではなく事業上等ということで事業を移すことがあったんです。実際にそれをやる際に従業員の方に対して会事をして、その方たちの同意を得ないと実行ができないということがありました。初めての案件なので自信もない中で従業員の方に説明をして、普通は5分とか10分くらいで終わると聞いていたんですが40分くらいの

In [22]:
# このままでも良いですが、さらに用途に合わせて整形してみましょう。
# 「議論の目的」「結論」を抽出してみます。
def ai_summary_v2(text):
    response = openai.ChatCompletion.create(
        engine="gpt-4o",
        response_format={ "type": "json_object" },
        messages=[
            {"role": "system", "content": "下記の文章は文字起こしのテキストファイルです。\
            「議論の目的」と「結論」を抽出し、それぞれ50字程度で要約し、下記を例にjson形式で出力してください。\
            *例　{'議論の目的':〇〇, '結論':○○}"},
            {"role": "user", "content": text}
        ],
    )
    # 回答の表示
    res = response.choices[0].message.content
    res = res.replace(" ", "").replace("\n", "")
    return res

res = ai_summary_v2(res)
res

'{"議論の目的":"田原信仰さんの仕事経験とエマンドエーセンター入社の理由を聞く","結論":"顧客に対する貢献とやりがいを感じており、今後も多くの案件を手掛けたい"}'

これで録画データ→データの整形まで完了しました。<br>
ここまでをまとめると、ライブラリという機能を組み合わせて、議事録アプリを作ることができました。
さらに、データベースへの格納をイメージし、これを表形式で整理してみましょう。

In [23]:
!pip install pandas
import pandas as pd
import json
# ここでのas pdは呼び名のようなもので、世間一般的にpdと定義されることが多いです。
# pandasは表計算のライブラリとして有名で、データサイエンスの領域でも多く用いられています。

In [24]:
# 先ほどのデータをjsonファイルに変換し、pandasで表にしてみます。
data_dict = json.loads(res)
df = pd.DataFrame([data_dict])
df

,議論の目的,結論
0,田原信仰さんの仕事経験とエマンドエーセンター入社の理由を聞く,顧客に対する貢献とやりがいを感じており、今後も多くの案件を手掛けたい


In [25]:
# 今回はデータベースではなく、csvで保存してみましょう。
df.to_csv("summary.csv", index=False)

保存後、左のファイルからダウンロードして中身を見てみましょう！<br>
新しい議事録が増えれば同じように出力しても良いですし、データベースに追記をするという処理によってデータを蓄積しても良いですね！

# streamlitでアプリに。
Pythonだけでwebアプリが作れる、streamlitというフレームワークで、これまで作ったコードを「アプリ」にしてみましょう。

In [26]:
!pip install streamlit
import streamlit as st

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 75.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.9 MB/s eta 0:00:00


In [38]:
%%writefile app.py

import streamlit as st
import openai
import pandas as pd
import whisper
import uuid
import os
import json
from moviepy.editor import VideoFileClip

# Azureの認証情報を設定
openai.api_type = "azure"
openai.api_base = "https://tech0-gpt-event-westus.openai.azure.com"
openai.api_key = "8db5e51a08ab497a9e31003e67b23a5d"
openai.api_version = "2023-05-15"

def ai_summary_v2(text):
    response = openai.ChatCompletion.create(
        engine="gpt-4o",
        response_format={ "type": "json_object" },
        messages=[
            {"role": "system", "content": "下記の文章は文字起こしのテキストファイルです。\
            「議論の目的」と「結論」を抽出し、それぞれ50字程度で要約し、下記を例にjson形式で出力してください。\
            *例　{'議論の目的':〇〇, '結論':○○}"},
            {"role": "user", "content": text}
        ],
    )
    # 回答の表示
    res = response.choices[0].message.content
    res = res.replace(" ", "").replace("\n", "")
    return res

def transcribe_audio_whisper(file_path):
    model = whisper.load_model("tiny")
    result = model.transcribe(file_path)
    return result["text"]

def main():
    st.title('超速作成議事録君')

    with st.spinner('ファイルを読み込み中...'):
        uploaded_file = st.file_uploader("ファイルを選択してください", type=['mp4', 'wav', 'm4a'])

    if uploaded_file is not None:
        file_extension = os.path.splitext(uploaded_file.name)[1]
        audio_file_name = uploaded_file.name.replace(file_extension, '.wav')

        with open(uploaded_file.name, "wb") as f:
            f.write(uploaded_file.getbuffer())

        if file_extension in ['.mp4', '.m4a']:
            clip = VideoFileClip(uploaded_file.name)
            clip.audio.write_audiofile(audio_file_name, 44100, 2, 2000, "pcm_s32le")
        else:
            audio_file_name = uploaded_file.name

        with st.spinner('文字起こし中...'):
            transcript_text = transcribe_audio_whisper(audio_file_name)

        if transcript_text:
            with st.spinner('要約中...'):
                res = ai_summary_v2(transcript_text)
                st.subheader("要約")
                data_dict = json.loads(res)
                df = pd.DataFrame([data_dict])
                st.write(df)
        else:
            st.error('文字起こしに失敗しました。')

if __name__ == "__main__":
    main()

Overwriting app.py


In [39]:
# ちょっとしたお作法
from google.colab import files
files.view("/content")
files.view("app.py")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [40]:
#streamlitをlocaltunnelのトンネルで起動
#実行画面に出るyour url is:のurlに接続し、ブラウザの別タブに移動
#Click to Continueでstreamlitの動作を確認
#ExternalURLのXXX.XXXの:よりも前の部分（●●.●●●.●●●.●●●）を入力しcommit
#確認が終わったらこのセルの実行を停止する
!streamlit run app.py & sleep 3 && npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.90.161.196:8501

your url is: https://true-ads-allow.loca.lt
error: XDG_RUNTIME_DIR not set in the environment.
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM default
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
